In [1]:
from math import log

# Linear Congruential Generator
# X(n+1) = (a * X(n) + c) mod m
class LCG:
    def __init__(self, seed=541, a=29, c=3, m=1289):
        # Initialize LCG with seed value and parameters
        self.seed = seed
        self.state = seed
        self.a = a
        self.c = c
        self.m = m

    def rand(self):
        # Generate next random number and normalize to [0, 1) range
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m

# Create a random number generator instance with default seed
rng = LCG()

In [2]:
# Exponential distribution
def exponential_var(mean, rng):
    # Formula: X = -mean * ln(1 - U) where U is uniform [0,1)
    return -log(1 - rng.rand()) * mean

# Generate uniform random variable in [low, high) range
def uniform_var(low, high, rng):
    return low + (high - low) * rng.rand()

In [3]:
# Generate inter-arrival times for customers with mean 8 minutes
def inter_arrival_time(hour, rng):
    times = []
    while sum(times) < hour * 60:
        times.append(round(exponential_var(8, rng), 2))
    return times

# Generate service times for customers
def service_time(customer_count, rng):
    service = []
    for _ in range(customer_count):
        # 30% chance: 1 person, 40% chance: 2 people, 20% chance: 3 people, 10% chance: 4 people
        r = rng.rand()
        if r < 0.3:
            party_size = 1
        elif r < 0.7:
            party_size = 2
        elif r < 0.9:
            party_size = 3
        else:
            party_size = 4

        # Service time: 25% chance of uniform(2,7), 75% chance of uniform(2,4)
        if rng.rand() < 0.25:
            service.append((round(uniform_var(2, 7, rng), 2), party_size))
        else:
            service.append((round(uniform_var(2, 4, rng), 2), party_size))
        # Returns tuple: (service_time, party_size)
    return service

In [4]:
# Generate inter-arrival times for 0.5 hours (30 minutes)
inter = inter_arrival_time(.5, rng)
# Generate service times for the same number of customers
service = service_time(len(inter), rng)

In [5]:
# Discrete event simulation setup
queue = []  # Queue to hold waiting customers
events = [(1, 0, "arrival")]  # Event list: (customer_id, time, event_type)
is_service_available = True  # Track if service is available
output = []  # Store all events for output

# Process events in chronological order
while len(events):
    # Get next event (earliest time)
    customer, time, event = events.pop(0)
    time = round(time, 2)
    output.append((customer, time, event))

    if event == "arrival":
        # Schedule next customer arrival if more customers exist
        if customer < len(inter):
            events.append((customer+1, time + inter[customer-1], "arrival"))
            
        if is_service_available:
            # If service is available, start service immediately
            is_service_available = False
            events.append((customer, time + service[customer-1][0], "departure"))
        else:
            # Otherwise, add customer to queue
            queue.append(customer)

    elif event == "departure":
        # Service completed, service becomes available
        is_service_available = True
        # If queue is not empty, start serving next customer
        if len(queue):
            next_customer = queue.pop(0)
            events.append((next_customer, time + service[next_customer-1][0], "departure"))
            is_service_available = False
    
    # Sort events by time (and event type for tie-breaking)
    events.sort(key=lambda x: (x[1], x[2]))


In [6]:
# Display simulation results
print(f"Inter-arrival times: {inter}")
print(f"Service times: {service}\n")
for customer, time, event in output:
    print(f"Customer {customer} at {time} {event}")

Inter-arrival times: [1.53, 0.34, 1.96, 2.87, 11.22, 16.35]
Service times: [(3.21, 1), (2.68, 1), (3.05, 3), (2.75, 1), (3.3, 4), (3.75, 3)]

Customer 1 at 0 arrival
Customer 2 at 1.53 arrival
Customer 3 at 1.87 arrival
Customer 1 at 3.21 departure
Customer 4 at 3.83 arrival
Customer 2 at 5.89 departure
Customer 5 at 6.7 arrival
Customer 3 at 8.94 departure
Customer 4 at 11.69 departure
Customer 5 at 14.99 departure
Customer 6 at 17.92 arrival
Customer 6 at 21.67 departure


In [7]:
# Full simulation function for 8.5 hours of operation
def simulate(rng):
    inter = inter_arrival_time(8.5, rng)
    service = service_time(len(inter), rng)

    queue = []
    events = [(1, 0, "arrival")]  # First customer arrives at time 0
    is_service_available = True
    output = []

    while len(events):
        customer, time, event = events.pop(0)
        time = round(time, 2)
        output.append((customer, time, event))

        if event == "arrival":
            if customer < len(inter):
                events.append((customer+1, time + inter[customer-1], "arrival"))
                
            if is_service_available:
                is_service_available = False
                events.append((customer, time + service[customer-1][0], "departure"))
            else:
                queue.append(customer)

        elif event == "departure":
            is_service_available = True
            if len(queue):
                next_customer = queue.pop(0)
                events.append((next_customer, time + service[next_customer-1][0], "departure"))
                is_service_available = False
        
        events.sort(key=lambda x: (x[1], x[2]))

    # Extract arrival and departure times for each customer
    # customers_events[i] = [arrival_time, departure_time]
    customers_events = [[-1, -1] for _ in range(len(inter)-1)]
    for customer, time, event in output:
        if customer == len(inter):
            continue
        if event == "arrival":
            customers_events[customer-1][0] = time
        elif event == "departure":
            customers_events[customer-1][1] = time

    # Calculate customer metrics
    customers_data = []
    for customer in range(len(customers_events)):
        # "Lucky" customers are those who depart before 8.5 hours (510 minutes)
        # Waiting time = (departure - arrival - service_time) * party_size
        customers_data.append(
            {
                "is_lucky": customers_events[customer][1] < (8.5 * 60),
                "customer_count": service[customer][1],
                "service_time": service[customer][0],
                "waiting_in_queue": (customers_events[customer][1] - customers_events[customer][0] - service[customer][0]) * service[customer][1],
            }
        )

    # Calculate statistics
    # Total lucky customers (sum of customer counts for lucky customers)
    lucky_count = sum(customer["customer_count"] if customer["is_lucky"] else 0 for customer in customers_data)
    # Total unlucky customers
    unlucky_count = sum(data[1] for data in service) - lucky_count

    # Average waiting time for lucky customers
    average_lucky_waiting = sum(customer["waiting_in_queue"] if customer["is_lucky"] else 0 for customer in customers_data) / lucky_count
    # Average waiting time for unlucky customers
    if unlucky_count:
        average_unlucky_waiting = sum(customer["waiting_in_queue"] if not customer["is_lucky"] else 0 for customer in customers_data) / unlucky_count
    else:
        average_unlucky_waiting = 0
    
    # Personnel utility: fraction of time service is busy (for lucky customers only)
    personnel_utility = sum(customer["service_time"] for customer in customers_data if customer["is_lucky"]) / (8.5 * 60)

    # Average ticket queue length: average number of customers waiting over time
    average_ticket_queue_length = sum(customer["waiting_in_queue"]*customer["customer_count"] for customer in customers_data) / (8.5 * 60)

    return lucky_count, unlucky_count, average_lucky_waiting, average_unlucky_waiting, personnel_utility, average_ticket_queue_length


In [8]:
# Run simulation for 7 days and calculate averages
def average_simulation(rng):
    # Initialize for statistics
    lucky_count, unlucky_count, average_lucky_waiting, average_unlucky_waiting, personnel_utility, average_ticket_queue_length = 0, 0, 0, 0, 0, 0
    
    # Run simulation for 7 days
    for _ in range(7):
        l, u, a, b, c, d = simulate(rng)
        # Sum results across all days
        lucky_count += l
        unlucky_count += u
        average_lucky_waiting += a
        average_unlucky_waiting += b
        personnel_utility += c
        average_ticket_queue_length += d

    # Print averaged results
    print(f"7 day simulation results with seed {rng.seed}:\n")
    print(f"Average lucky count: {round(lucky_count / 7, 2)} customers")
    print(f"Average unlucky count: {round(unlucky_count / 7, 2)} customers")
    print(f"Average lucky waiting: {round(average_lucky_waiting / 7, 2)} minutes")
    print(f"Average unlucky waiting: {round(average_unlucky_waiting / 7, 2)} minutes")
    print(f"Personnel utility: {round(personnel_utility / 7, 2)} customers")
    print(f"Average ticket queue length: {round(average_ticket_queue_length / 7, 2)} customers")


In [9]:
# Run 7-day simulation with seed 542
average_simulation(LCG(seed=542))

7 day simulation results with seed 542:

Average lucky count: 123.0 customers
Average unlucky count: 2.0 customers
Average lucky waiting: 1.01 minutes
Average unlucky waiting: 0.0 minutes
Personnel utility: 0.39 customers
Average ticket queue length: 0.59 customers


In [10]:
# Run 7-day simulation with seed 543
average_simulation(LCG(seed=543))

7 day simulation results with seed 543:

Average lucky count: 124.29 customers
Average unlucky count: 2.57 customers
Average lucky waiting: 0.89 minutes
Average unlucky waiting: 1.0 minutes
Personnel utility: 0.37 customers
Average ticket queue length: 0.6 customers


In [11]:
# Run 7-day simulation with seed 544
average_simulation(LCG(seed=544))

7 day simulation results with seed 544:

Average lucky count: 124.0 customers
Average unlucky count: 3.43 customers
Average lucky waiting: 2.26 minutes
Average unlucky waiting: 0.77 minutes
Personnel utility: 0.42 customers
Average ticket queue length: 1.37 customers


In [12]:
# Run 7-day simulation with seed 545
average_simulation(LCG(seed=545))

7 day simulation results with seed 545:

Average lucky count: 131.43 customers
Average unlucky count: 2.43 customers
Average lucky waiting: 0.93 minutes
Average unlucky waiting: 0.0 minutes
Personnel utility: 0.4 customers
Average ticket queue length: 0.61 customers
